In [ ]:
# Import neccesary programs
import requests
import bs4
from bs4 import BeautifulSoup
from lxml import html
import numpy as np
import pandas as pd

### Webscraping Functions

In [ ]:
# function for getting location from indeed

def extract_location_from_result(loc):
    try:
        for n in loc.findAll(class_='location'):
            return n.text.strip()
    except:
        return 'None'


In [ ]:
# function for getting company from indeed

def extract_company_from_result(comp):
    try:
        for n in comp.findAll(class_='company'):
            return n.text.strip()
    except:
        return 'None' 

In [ ]:
# function for getting job title from indeed

def extract_jobtitle_from_result(jobtitle):
    try:
        for n in jobtitle.findAll(class_='jobtitle'):
            return n.text.strip()
    except:
        return 'None'   

In [ ]:
# function to get salary from indeed

def extract_salary_from_result(sal):
    try:
        for n in sal.findAll('td', class_='snip'):
            return n.nobr
    except:
        return 'None' 

In [ ]:
# function to get summary/description from indeed

def extract_summary_from_result(desc):
    try:
        for n in desc.findAll('span', class_='summary'):
            return n.text.strip()
    except:
        return 'None' 

### Select Website 

In [ ]:
Location = []
Job_Title = []
Company = []
Salary = []
Description = []

url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 100
cities = ['New+York', 'Chicago', 'Boston', 'San+Francisco', 'Los+Angeles', 'Austin', 'Atlanta']
results = []
for city in set(cities):
    for start in range(0, max_results_per_city, 10):
        url = url_template.format(city,start)
        ru = requests.get(url)
        indeed = BeautifulSoup(ru.content, "lxml")
        for item in indeed.findAll('div', class_='result'):
            Location.append(extract_location_from_result(item))
            Job_Title.append(extract_jobtitle_from_result(item))
            Company.append(extract_company_from_result(item))
            Salary.append(extract_salary_from_result(item))
            Description.append(extract_summary_from_result(item))

In [ ]:
# Create dataframe and name columns. Include Cities
df = pd.DataFrame([Location,Job_Title,Company,Salary,Description]).T 
  
df.columns= ['Location','Job_Title','Company','Salary','Description']

In [ ]:
df['City'] = lambda x: pd.series([c for c in ('Location')(c.split(','))])


In [ ]:
df = pd.DataFrame(df.row.str.split(' ',1).tolist(),
                                   columns = )

In [1]:
df = pd.DataFrame(df['Location'.str.split(' ',1).tolist(),
                                   columns = ['Cities','Location'])

SyntaxError: invalid syntax (<ipython-input-1-dd952af3daa7>, line 2)

In [ ]:
df.head()

In [ ]:
df.dtypes

### Cleaning Salary Data

In [ ]:
df['Salary'].unique()

In [ ]:
# Remove rows with no salary
df = df[df.Salary.notnull()]

In [ ]:
# Remove excess salary info
df['Salary'] = df['Salary'].apply(lambda x: str(x))
df['Salary'] = df['Salary'].apply(lambda x: x.replace('$',''))
df['Salary'] = df['Salary'].apply(lambda x: x.replace(',',''))
df['Salary'] = df['Salary'].apply(lambda x: x.replace('<nobr>',''))
df['Salary'] = df['Salary'].apply(lambda x: x.replace('</nobr>',''))



In [ ]:
# Reset index

df = df.reset_index(drop=True)

In [ ]:
# Parse yearly, monthly, hourly salaries
df['hourly_sal'] = df['Salary'].map(lambda x: 1 if 'hour' in x else 0)

df['monthly_sal'] = df['Salary'].map(lambda x: 1 if 'month' in x else 0)

df['yearly_sal'] = df['Salary'].map(lambda x: 1 if 'year' in x else 0)



In [ ]:
# Drop monthly and hourly salaries
df_year1 = df[df.monthly_sal != 1]
df_year = df_year1[df.hourly_sal !=1]


In [ ]:
df_year.head()

In [ ]:
# drop columns hourly_sal, monthly_sal, and yearly_sal
df_year = df_year.drop('hourly_sal', axis=1)
df_year = df_year.drop('monthly_sal', axis=1)
df_year = df_year.drop('yearly_sal', axis=1)

In [ ]:
# Check for unique Salary values
df_year['Salary'].unique

In [ ]:
# reset index and set dataframe to df
df = df_year.reset_index(drop=True)

In [ ]:
# Remove non-numeric Salary elements
df['Salary'] = df['Salary'].str.replace('a year', '')
df['Salary'] = df['Salary'].str.replace('-', ',')


In [ ]:
# check for other non-numeric values
df['Salary'].unique()

In [ ]:
# remove '1250 a week'

df = df[df.Salary.str.contains('1250') == False]
df['Salary'].unique()

### Running Predictions

In [ ]:

df = pd.get_dummies(df,columns = ['Location','Job_Title','Company'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 77) ## create train-test out of the data given
""" Fit a binary classification predictor."""
logreg = LogisticRegression(solver='liblinear')
C_vals = [0.0001, 0.001, 0.01, 0.1, .15, .25, .275, .33, 0.5, .66, 0.75, 1.0, 2.5, 5.0, 10.0, 100.0, 1000.0]
penalties = ['l1','l2']

gs = GridSearchCV(logreg, {'penalty': penalties, 'C': C_vals}, verbose=False, cv=3)
gs.fit(X_train, y_train)

lr = LogisticRegressionCV(Cs=10, cv=5)
cv_model = logreg.fit(X_train, y_train)
cv_pred = cv_model.predict(X_test)

y_pred = lr.predict(X_test)
y_score = lr.decision_function(X_test) # Submit these response, the output of mode

In [ ]:
conmat = np.array(confusion_matrix(Y_test, y_pred, labels=[1,0]))
confusion = pd.DataFrame(conmat, index=['over_50k', 'under_50k'],
                            columns=['predicted_over50k','predicted_under50k'])

print(confusion)
print classification_report(Y_test,y_pred)
roc_auc_score(Y_test, y_score)

In [ ]:
coef = pd.DataFrame([X.columns.values.tolist(), lr.coef_[0].tolist()], index=['features', 'coef']).T
coef.sort_values(by='coef', ascending=False)